In [39]:
import pandas as pd
import time
from time import time
import os

import os
import gensim
from gensim.models import Word2Vec, KeyedVectors

from nltk.corpus import stopwords
from nltk import download
stop_words = stopwords.words('english')
from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

from gensim.similarities import WmdSimilarity

start_nb = time()
#https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html

[nltk_data] Downloading package punkt to /Users/whs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
# Pre-processing a document.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

In [9]:
relPath = 'data/1.0-billTitleSponsors.csv'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

'/Users/whs/Documents/GitHub/loc_challenge/data/1.0-billTitleSponsors.csv'

In [10]:
#data_dir = '/Users/whs/Documents/GitHub/gpo-ai.com/data/processed/1.0-billTitleSponsors.csv'
#data_dir = 'C:\\Users\\wsolomon\\Desktop\\DataScience\\projects\\loc_challenge-master\\1.0-billTitleSponsors.csv'
relPath = 'data/1.0-billTitleSponsors.csv'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)
df = pd.read_csv(absPath)
df.head()

,Unnamed: 0,title,sponsers,texts
0,1,Tax Cuts and Jobs Act,"[Sen. Enzi, Michael B. [R-WY]]",This bill amends the Internal Revenue Code to ...
1,11,Jerusalem Embassy and Recognition Act,"[Sen. Heller, Dean [R-NV], Sen. Cruz, Ted [R-T...",This bill states that it should be U.S. polic...
2,12,Increasing the Department of Veterans Affairs ...,"[Sen. Moran, Jerry [R-KS], None, Sen. Moran, J...",This bill requires the reduction of the feder...
3,13,No Windfalls for Government Service Act,"[Sen. Whitehouse, Sheldon [D-RI], Sen. Baldwin...",This bill amends the Internal Revenue Code to ...
4,14,"No Budget, No Pay Act","[Sen. Heller, Dean [R-NV], Sen. Manchin, Joe, ...",This bill prohibits the payment of any pay to ...


In [11]:
start = time()

relPath = 'model/GoogleNews-vectors-negative300.bin.gz'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

#path = '/Users/whs/Documents/GitHub/gpo-ai.com/data/processed/GoogleNews-vectors-negative300.bin.gz'
if not os.path.exists(absPath):
    raise ValueError("SKIP: You need to download the google news model")
    
model = gensim.models.KeyedVectors.load_word2vec_format(absPath, binary=True)

print('Cell took %.2f seconds to run.' % (time() - start))

Using TensorFlow backend.


Cell took 268.99 seconds to run.


In [12]:
doc1 = df['texts'][3]
doc2 = df['texts'][4]
distance = model.wmdistance(doc1,doc2)
print('distance = %.4f' % distance)

distance = 0.5135


In [25]:
print(df.shape)
df = df.dropna()
df.shape

(7426, 4)


(5663, 4)

In [26]:
#preprocess(df.texts.values[1000])
df.texts.values[1000]

'This bill amends the Equal Credit Opportunity Act (ECOA) to expand the categories of prohibited discrimination when extending credit to include discrimination on account of sexual orientation or gender identity. (Currently, such discrimination in credit is prohibited only on the basis of race, color, religion, national origin, sex or marital status, or age.)'

In [29]:
start = time()

wmd_corpus = [preprocess(doc) for doc in df.texts.values]
print('Docs in Corpus: ',len(wmd_corpus))
num_best = 4
instance = WmdSimilarity(wmd_corpus, model, num_best=num_best)

print('Cell took %.2f seconds to run.' % (time() - start))

Docs in Corpus:  5663
Cell took 63.10 seconds to run.


In [30]:
start = time()

sent = 'guns'
query = preprocess(sent)

sims = instance[query]  # A query is simply a "look-up" in the similarity class.
print(sims)
#print('Cell took %.2f seconds to run.' % (time() - start))

# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sent)
for i in range(num_best):
    print()
    print('sim = %.4f' % sims[i][1])
    print(df.title.values[sims[i][0]])
    print(df.sponsers.values[sims[i][0]])

[(1446, 0.45725883121398953), (1182, 0.44884097239488163), (1032, 0.4474319014548333), (4799, 0.44371549025724843)]
Query:
guns

sim = 0.4573
Handgun Trigger Safety Act of 2017
[Sen. Van Hollen, Chris [D-MD], None, Sen. Markey, Edward J. [D-MA]]

sim = 0.4488
Protecting the Second Amendment Act
[Sen. Strange, Luther [R-AL], Sen. Cruz, Ted [R-TX], Sen. Cornyn, John [R-TX]]

sim = 0.4474
Stop Illegal Trafficking in Firearms Act of 2017
[Sen. Durbin, Richard J. [D-IL], None, Sen. Leahy, Patrick J. [D-VT]]

sim = 0.4437
Protecting Domestic Violence and Stalking Victims Act of 2013
[None, Sen. Klobuchar, Amy [D-MN]]


In [36]:
relPath = 'model/1.0-whs-W2VModel'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

model.save(absPath)

In [33]:
relPath = 'model/1.1-whs-Sim_Instance'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

instance.save(absPath)

In [42]:
w2v_relPath = 'model/GoogleNews-vectors-negative300.bin.gz'
w2v_absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

relPath = 'model/1.0-whs-W2VModel'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)
#modelX = KeyedVectors.load_word2vec_format(w2v_absPath, binary=True) #.load(absPath)

relPath = 'model/1.1-whs-Sim_Instance'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)
instanceX = WmdSimilarity.load(absPath)

relPath = 'data/1.0-billTitleSponsors.csv'
absPath = os.path.join(os.path.dirname(os.getcwd()), relPath)

df = pd.read_csv(absPath)

In [46]:
start = time()

sent = 'I like sports'
query = preprocess(sent)

sims = instanceX[query]  # A query is simply a "look-up" in the similarity class.
print(sims)
#print('Cell took %.2f seconds to run.' % (time() - start))

# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sent)
for i in range(num_best):
    print()
    print('sim = %.4f' % sims[i][1])
    print(df.title.values[sims[i][0]])
    print(df.sponsers.values[sims[i][0]])


print('Cell took %.2f seconds to run.' % (time() - start))

[(1202, 0.45851386949294604), (3798, 0.44686855582026991), (4965, 0.44571707949836059), (3193, 0.44310248357413906)]
Query:
I like sports

sim = 0.4585
United States Center for Safe Sport Authorization Act of 2017
[Sen. Thune, John [R-SD], Sen. Nelson, Bill [D-FL], Sen. Moran, Jerry [R-KS], Sen. Peters, Gary C. [D-MI], Sen. Sullivan, Dan  [R-AK], Sen. Klobuchar, Amy [D-MN], Sen. Blumenthal, Richard [D-CT]]

sim = 0.4469
FANS Act
[None, Sen. Blumenthal, Richard [D-CT]]

sim = 0.4457
PRO Sports Act
[Sen. Coburn, Tom [R-OK], Sen. King, Angus S. Jr. [I-ME]]

sim = 0.4431
S. 1761, a bill to take certain federal land located in Lassen County, California, into trust for the benefit of the Susanville Indian Rancheria, and for other purposes
[None, Sen. Boxer, Barbara [D-CA]]
Cell took 19.20 seconds to run.


In [32]:
start = time()

modelX = Word2Vec.load(path)
from gensim.similarities import WmdSimilarity
wmd_corpus = [preprocess(doc) for doc in df.title.values]
print(len(wmd_corpus))
num_best = 10
instanceX = WmdSimilarity(wmd_corpus, modelX, num_best=10)


print('Cell took %.2f seconds to run.' % (time() - start))

NameError: name 'path' is not defined

In [35]:
start = time()

#model = Word2Vec.load(fname)
path = '/Users/whs/Documents/GitHub/gpo-ai.com/models/1.1-whs-Sim_Instance'
instanceX = WmdSimilarity.load(path)
sent = 'guns'
query = preprocess(sent)

sims = instanceX[query]  # A query is simply a "look-up" in the similarity class.
print(sims)
#print('Cell took %.2f seconds to run.' % (time() - start))

# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sent)
for i in range(num_best):
    print()
    print('sim = %.4f' % sims[i][1])
    print(df.title.values[sims[i][0]])
    print(df.sponsers.values[sims[i][0]])

[(54, 0.43357007474150921), (82, 0.43357007474150921), (5, 0.43197784960781815), (70, 0.43141103399568781), (26, 0.43080101983513447), (77, 0.42987584221802178), (20, 0.42965395615410074), (31, 0.428680079458092), (41, 0.42692412452172734), (2, 0.4268953577652228)]
Query:
guns

sim = 0.4336
Safe and Affordable Drugs from Canada Act of 2017
['Sen. McCain, John [R-AZ]']

sim = 0.4336
Safe and Affordable Drugs from Canada Act of 2017
['Sen. Grassley, Chuck [R-IA]', 'Sen. Leahy, Patrick J. [D-VT]', 'Sen. King, Angus S., Jr. [I-ME]', 'Sen. Whitehouse, Sheldon [D-RI]', 'Sen. Brown, Sherrod [D-OH]', 'Sen. Baldwin, Tammy [D-WI]', 'Sen. Shaheen, Jeanne [D-NH]', 'Sen. Merkley, Jeff [D-OR]', 'Sen. Hassan, Margaret Wood [D-NH]', None, 'Sen. McCain, John [R-AZ]']

sim = 0.4320
Iran Ballistic Missile Sanctions Act
['Sen. Heller, Dean [R-NV]']

sim = 0.4314
A bill to protect the right of individuals to bear arms at water resources development projects.
['Sen. Boozman, John [R-AR]', None, 'Sen. Crapo,